# Exportacion de Modelo y Contrato de Integracion

Objetivo: exportar el modelo y documentar el pipeline de inferencia para back-end.


## 1. Exportar el modelo (joblib)


In [1]:
import json
from pathlib import Path
import joblib

src_model = Path('..') / '04_ENTRENAMIENTO_Y_EVALUACION' / 'MODELO_ENTRENADO_V1' / 'model.joblib'
dst_model = Path('outputs') / 'model.joblib'
dst_model.parent.mkdir(parents=True, exist_ok=True)

model = joblib.load(src_model)
joblib.dump(model, dst_model)
dst_model


WindowsPath('outputs/model.joblib')

## 2. Crear metadata del modelo


In [2]:
metadata = {
    'model_type': 'LogisticRegression',
    'version': 'V1',
    'features': {
        'categorical': ['aerolinea', 'origen', 'destino'],
        'numerical': ['distancia_km', 'dep_hour', 'dep_dow', 'dep_month', 'temp', 'wind_spd', 'precip_1h', 'climate_severity_idx'],
        'target': 'DEP_DEL15',
    },
    'inference_time': 'T-2h',
    'notes': 'Clima obtenido via API externa con pronostico disponible 2 horas antes.'
}

Path('outputs').mkdir(parents=True, exist_ok=True)
Path('outputs') / 'metadata.json'
(Path('outputs') / 'metadata.json').write_text(json.dumps(metadata, indent=2), encoding='utf-8')


481

## 3. Crear example_request.json


In [3]:
example_request = {
  'aerolinea': 'AZ',
  'origen': 'GIG',
  'destino': 'GRU',
  'fecha_partida': '2025-11-10T14:30:00',
  'distancia_km': 350
}
(Path('outputs') / 'example_request.json').write_text(json.dumps(example_request, indent=2), encoding='utf-8')


127

## 4. Crear example_response.json


In [4]:
example_response = {
  'prevision': 'Puntual',
  'probabilidad': 0.65
}
(Path('outputs') / 'example_response.json').write_text(json.dumps(example_response, indent=2), encoding='utf-8')


52

## 5. Documentar pipeline de inferencia


In [5]:
docs_path = Path('docs')
docs_path.mkdir(parents=True, exist_ok=True)

contrato = '''# Contrato de Integracion

## Request
```json
{
  "aerolinea": "AZ",
  "origen": "GIG",
  "destino": "GRU",
  "fecha_partida": "2025-11-10T14:30:00",
  "distancia_km": 350
}
```

## Response
```json
{
  "prevision": "Puntual|Retrasado",
  "probabilidad": 0.00
}
```

## Enriquecimiento de clima
- Se consulta un servicio externo con latitud/longitud y fecha_partida.
- Se usa el pronostico disponible 2 horas antes de la salida.
- Variables: TEMP, WIND_SPD, PRECIP_1H, CLIMATE_SEVERITY_IDX.
'''

( docs_path / 'contrato_integracion.md').write_text(contrato, encoding='utf-8')

pipeline = '''# Pipeline de Inferencia

1. Recibir JSON de entrada (contrato fijo).
2. Enriquecer con clima via API externa (lat/lon + fecha_partida).
3. Construir features:
   - aerolinea, origen, destino
   - distancia_km
   - dep_hour, dep_dow, dep_month (desde fecha_partida)
   - temp, wind_spd, precip_1h, climate_severity_idx
4. Cargar modelo joblib y predecir probabilidad.
5. Mapear salida a {prevision, probabilidad}.
'''

( docs_path / 'pipeline_inferencia.md').write_text(pipeline, encoding='utf-8')


414